# K-modes

**Problème avec le K-means:** le K-Means ne peuvent pas gérer les données catégoriques, car il minimise explicitement la variance intra-cluster (distances au carré de la moyenne) comme défini dans l'espace euclidien. 

**=> le K-modes comme alternative**

Tandis que le K-Means calcule la distance euclidienne entre deux points, le K-Modes tente de minimiser une mesure de dissimilarité: il compte le nombre de "features" qui ne sont pas les mêmes. En utilisant des modes au lieu de moyens, K-Modes devient capable de gérer efficacement des données catégorielles.

## Algorithme:

**Définition du mode cluster:** si un jeu de données possède m attributs catégoriels, le vecteur mode Z est constitué de m valeurs catégorielles, chacune étant le mode d'un attribut.

1. Sélectionnez k modes initiaux, un pour chaque cluster.
2. Allouez un objet au cluster dont le mode est le plus proche en fonction de la distance de Hamming (La distance de Hamming entre deux lignes est simplement le nombre de colonnes où les deux lignes diffèrent). Mettez à jour le mode du cluster après chaque allocation.
3. Une fois que tous les objets ont été attribués aux clusters, testez à nouveau la dissemblance des objets par rapport aux modes actuels. Si un objet est trouvé de telle sorte que son mode le plus proche appartient à un autre cluster plutôt que son cluster actuel, réattribuez l'objet à ce cluster et mettez à jour les modes des deux clusters.
4. Répétez 3 jusqu'à ce qu'aucun objet n'ait changé de clusters après un test de cycle complet de l'ensemble de données.

## K-modes vs K-means + one-hot encoding:
- Kmeans + one-hot encoding augmentera considérablement la taille de l'ensemble de données si les attributs catégoriels ont un grand nombre de catégories. Cela rendra le K-means coûteuses en calcul.
- Le cluster signifie n'a pas de sens puisque le 0 et le 1 ne sont pas les valeurs réelles des données. Le K-modes, d'autre part, produisent des modes de cluster qui sont les données réelles et rendent donc les clusters interprétables.

## Inconvénients du K-modes:
- Lorsque les types de données sont mélangés
- Il compte simplement le nombre de dissemblances mais ne considère pas quels "features" sont différents.


In [1]:
"""
KModes
"""
#imports
import time
import numpy as np
from scipy import stats

class K_modes:
    def __init__(self,n_clusters):
        self.n_clusters = n_clusters
        self.modes = []
        self.labels = []

    def fit(self,X_train):
        size = X_train.shape
        self.labels = [0]*size[0]
        #random selection of culster modes
        modes = np.random.choice(size[1], self.n_clusters, replace=False)
        not_stable = True
        for element in modes:
            self.modes.append(X_train[element])
        

        while not_stable : 
            distance = np.empty((0,size[0]), int)
            for current_mode in self.modes:
                #print((X_train-current_mode!=0).sum(axis=1).shape)
                distance = np.append(distance,(X_train-current_mode!=0).sum(axis=1).reshape(1,-1),axis=0)
            #print("distance is",distance)
            self.labels= distance.argmin(axis=0)
            
            
            #calculate the distances between the modes and all the individuals
            '''for i in range (0,size[1]):
                distance = [0]*self.n_clusters
                for k in range (0,self.n_clusters):
                    for j in range (0,size[1]):
                        distance[k] = (distance[k] + 1) if X_train[i][j] != self.modes[k][j] else distance[k]

            # assign the individual to the cluster with minimum distance 
                self.labels[i] = distance.index(min(distance))'''

            #keep the old modes to compare later
            modes_old = self.modes[:]
            print(self.labels)
            #claculate the new modes 
            for i in range (0,self.n_clusters):
                self.modes[i] = stats.mode(X_train[np.array(self.labels) == i])[0][0]
                #print(self.modes[i])
            
            #check if changes occured to the modes
            not_stable = False 
            for i in range (0,len(self.modes)):
                print("array",self.modes[i])
                if not(np.array_equal(self.modes[i],modes_old[i])):
                    not_stable =  True

        return self




#test data 
data = np.array([[1,2,1,2,3],[1,1,1,2,2],
                [3,1,2,2,1],[1,2,2,1,3],
                [3,3,3,2,1],[1,1,1,1,2],
                [1,3,1,3,3],[3,1,2,2,3],
                [1,1,2,3,1],[1,2,2,1,3]])


start_time = time.time()
k = K_modes(3)
k.fit(data)
print(k.modes)
print(k.labels)
print("--- %s seconds ---" % (time.time() - start_time))

[1 1 2 0 2 0 1 2 2 0]
array [1 2 2 1 3]
array [1 1 1 2 3]
array [3 1 2 2 1]
[1 1 2 0 2 1 1 2 2 0]
array [1 2 2 1 3]
array [1 1 1 2 2]
array [3 1 2 2 1]
[0 1 2 0 2 1 0 2 2 0]
array [1 2 1 1 3]
array [1 1 1 1 2]
array [3 1 2 2 1]
[0 1 2 0 2 1 0 2 2 0]
array [1 2 1 1 3]
array [1 1 1 1 2]
array [3 1 2 2 1]
[array([1, 2, 1, 1, 3]), array([1, 1, 1, 1, 2]), array([3, 1, 2, 2, 1])]
[0 1 2 0 2 1 0 2 2 0]
--- 0.010764837265014648 seconds ---


In [ ]:
#test sur la BD des iris
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head()
iris=np.array(iris)
sns.load_dataset('iris')

In [2]:
start_time = time.time()
k = K_modes(5)
k.fit(iris)
print(k.modes)

print("--- %s seconds ---" % (time.time() - start_time))
print(k.labels)

NameError: name 'iris' is not defined

In [ ]:
#test Kmeans comparer les temps d'execution
from sklearn.cluster import KMeans
start_time = time.time()
kmeans = KMeans(n_clusters=5, random_state=0).fit(iris)

print("--- %s seconds ---" % (time.time() - start_time))
kmeans.labels_